<a href="https://colab.research.google.com/github/programminghunk/sugarcane/blob/main/experiment_custom_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/ultralytics/ultralytics.git

In [ ]:
#%cd ultralytics
#!pip install -e '.[dev]'

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.8/8062.4 GB disk)


In [ ]:
%cd ultralytics/cfg/models/v8

/kaggle/working/ultralytics/ultralytics/cfg/models/v8


In [ ]:
%%writefile newconf.yaml

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  - [-1, 1, Conv, [16, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [32, 3, 2]] # 1-P2/4
  - [-1, 1, ELAN1, [32, 32, 16]] # 2
  - [-1, 1, AConv, [64]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 4
  - [-1, 1, AConv, [96]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [96, 96, 48, 3]] # 6
  - [-1, 1, AConv, [128]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [128, 128, 64, 3]] # 8
  - [-1, 1, SPPELAN, [128, 64]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


Writing newconf.yaml


In [ ]:
# %load newconf.yaml

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  - [-1, 1, Conv, [16, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [32, 3, 2]] # 1-P2/4
  - [-1, 1, ELAN1, [32, 32, 16]] # 2
  - [-1, 1, AConv, [64]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 4
  - [-1, 1, AConv, [96]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [96, 96, 48, 3]] # 6
  - [-1, 1, AConv, [128]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [128, 128, 64, 3]] # 8
  - [-1, 1, SPPELAN, [128, 64]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


SyntaxError: invalid syntax (3766294738.py, line 5)

In [ ]:
%cd ..

/kaggle/working/ultralytics


In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO("/kaggle/working/ultralytics/ultralytics/cfg/models/v8/newconf.yaml")

# Display model information (optional)
model.info()

WARNING ⚠️ no model scale passed. Assuming scale='n'.
newconf summary: 515 layers, 2,212,808 parameters, 2,212,792 gradients, 7.1 GFLOPs


(515, 2212808, 2212792, 7.1403008)

In [ ]:
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("np_mynp")
version = project.version(2)
dataset = version.download("yolov8")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.72, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to np_mynp-2 in yolov8:: 100%|██████████| 6719/6719 [00:01<00:00, 6147.26it/s]


In [ ]:
# %load np_mynp-2/data.yaml
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/2
  version: 2
  workspace: buddetection
test: np_mynp-2/test/images
train: np_mynp-2/train/images
val: np_mynp-2/test/images


In [ ]:
%%writefile np_mynp-2/data.yaml
path: /kaggle/working/ultralytics
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/1
  version: 2
  workspace: buddetection
test: np_mynp-2/test/images
train: np_mynp-2/train/images
val: np_mynp-2/test/images

Overwriting np_mynp-2/data.yaml


In [ ]:
%pwd

'/kaggle/working/ultralytics'

In [ ]:
results = model.train(data="np_mynp-2/data.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/ultralytics/ultralytics/cfg/models/v8/newconf.yaml, data=np_mynp-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fa

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]
2024-08-04 03:13:01,820	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-04 03:13:03,129	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=2
WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       232  ultralytics.nn.modules.conv.Conv             [3, 8, 3, 2]                  
  1                  -1  1       592  ultralytics.nn.modules.conv.Conv             [8, 8, 3, 2]                  
  2                  -1  1      5520  ultralytics.nn.modules.block.ELAN1           [8, 8, 32, 16]                
  3                  -1  1      1184  ultralytics.nn.modules.block.AConv           [8, 16]                       
  4                  -1  1     55904  ultralytics.nn.modules.block.RepNCSPELAN4    [16, 16, 64, 32, 3]           
  5                  -1  1      3504  ultralytics.nn.modules.block.AConv           [16, 24]                      
  6                  -1  1    124944  ultralytics.nn.modules.block.RepNCSPELAN4    [24, 24, 96, 48, 3]       

100%|██████████| 6.25M/6.25M [00:00<00:00, 75.9MB/s]


AMP: checks passed ✅


...

    100/100      2.11G     0.7332     0.4715      1.133         22        640: 100%|██████████| 193/193 [00:33<00:00,  5.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.83it/s]

                   all        268        347      0.881      0.864      0.923      0.655



100 epochs completed in 1.051 hours.
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train/weights/last.pt, 4.5MB
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train/weights/best.pt, 4.5MB

Validating /kaggle/working/ultralytics/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
newconf summary (fused): 300 layers, 2,049,814 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.67it/s]


                   all        268        347       0.88      0.864      0.923      0.656
                   FIT        125        162      0.888      0.833      0.917      0.698
                 UNFIT        163        185      0.873      0.895      0.929      0.613
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /kaggle/working/ultralytics/runs/detect/train


In [ ]:
!zip -r outf.zip /kaggle/working/ultralytics/runs/detect/train

  adding: kaggle/working/ultralytics/runs/detect/train/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train/results.png (deflated 9%)
  adding: kaggle/working/ultralytics/runs/detect/train/train_batch2.jpg (deflated 1%)
  adding: kaggle/working/ultralytics/runs/detect/train/val_batch1_pred.jpg (deflated 4%)
  adding: kaggle/working/ultralytics/runs/detect/train/train_batch1.jpg (deflated 1%)
  adding: kaggle/working/ultralytics/runs/detect/train/val_batch1_labels.jpg (deflated 5%)
  adding: kaggle/working/ultralytics/runs/detect/train/train_batch17372.jpg (deflated 2%)
  adding: kaggle/working/ultralytics/runs/detect/train/train_batch0.jpg (deflated 0%)
  adding: kaggle/working/ultralytics/runs/detect/train/args.yaml (deflated 53%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/best.pt (deflated 13%)
  adding: kaggle/working/ultralytics/runs/detect/train/weights/last.pt (deflated 

In [ ]:
from IPython.display import FileLink
FileLink('outf.zip')

/kaggle/working/outf.zip

In [ ]:
!cp /kaggle/working/ultralytics/out.zip /kaggle/working/


In [ ]:
import os

# Define the file path
file_path = '/kaggle/working/ultralytics/out.zip'

# Check if the file exists and delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")

/kaggle/working/ultralytics/out.zip does not exist.


In [ ]:
!rm /kaggle/working/ultralytics/out1.zip

In [ ]:
%cd ..

/kaggle/working


In [ ]:
%ls

outf.zip  ultralytics/
